# Spark Configrations

In [1]:
from pyspark.sql import SparkSession, functions as F
import findspark
import pandas as pd
import boto3
from datetime import datetime

current_ts = datetime.now().strftime('%Y%m%d')

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', None)

from pyspark.sql.types import *
from pyspark.sql.functions import col, json_tuple, to_json, from_json, explode_outer

# /opt/manual/spark: this is SPARK_HOME path
findspark.init("/opt/manual/spark")

#spark = SparkSession.builder.config("spark.jars.packages","org.apache.hadoop:hadoop-aws:3.2.0").getOrCreate()
# spark & minio configuration on s3
spark = (SparkSession.builder
         .appName("Spark Minio Test")
         .config("spark.hadoop.fs.s3a.endpoint", "http://172.18.0.2:9000")
         .config("spark.hadoop.fs.s3a.access.key", 'root')
         .config("spark.hadoop.fs.s3a.secret.key", 'root12345')
         .config("spark.hadoop.fs.s3a.path.style.access", True)
         .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
         .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.0,io.delta:delta-core_2.12:1.0.0")
         # .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0")
         .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
         .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
         .getOrCreate())

# Delta ve s3a configlerinin ayni sparksession altinda calistirilmasi icin, config.jars.packages ',' ile birlestirilmeli!!!
# `.config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.0,io.delta:delta-core_2.12:1.0.0")` seklinde uygundur


:: loading settings :: url = jar:file:/opt/manual/spark-3.1.1-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/train/.ivy2/cache
The jars for the packages stored in: /home/train/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5bab3dc7-f2ee-4d39-b349-545e369a8b81;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.0 in spark-list
	found com.amazonaws#aws-java-sdk-bundle;1.11.375 in central
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 499ms :: artifacts dl 13ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.375 from central in [default]
	com.ibm.icu#icu4j;

In [2]:
from delta.tables import *

# BOTO3 Configrations

In [3]:
BUCKET = 'tmdb-bronze'
HTTP = 'http://'
ENDPOINT = 'localhost:9000'
AWS_ACCESS_KEY_ID = 'root'
AWS_SECRET_ACCESS_KEY = 'root12345'
REGIONNAME = 'whatever'

session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=None,
    region_name=REGIONNAME,
    botocore_session=None,
    profile_name=None)

s3client = session.client(
    's3', endpoint_url=f"{HTTP}{ENDPOINT}")

s3resource = session.resource(
    's3', endpoint_url=f"{HTTP}{ENDPOINT}")


my_bucket = s3resource.Bucket('tmdb-bronze')
file_names = []
# for my_bucket_object in my_bucket.objects.all():
#     key=my_bucket_object.key
#     if key.find(f"credits/credits_part_{current_ts}-") == 0:
#         file_names.append(key[-14:-8])
# file_names
bronze_bucket = s3resource.Bucket('tmdb-bronze')
silver_bucket = s3resource.Bucket('tmdb-silver')
current_ts = datetime.now().strftime('%Y%m%d')

# Read from Bronze

## 1. Credits parquet

In [4]:
file_names = []
for bronze_bucket_object in bronze_bucket.objects.all():
    key = bronze_bucket_object.key
    if key.find(f"credits/credits_part_{current_ts}-") == 0:
        file_names.append(key[-14:-8])

In [5]:
credits = (spark.read.format("parquet")
               .option("header", True)
               .option("inferSchema", True)
               .load(f"s3a://tmdb-bronze/credits/credits_part_{current_ts}-{file_names[0]}.parquet"))

In [6]:
credits.limit(5).toPandas()

/home/train/venvairflow/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:178: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

# Read from Silver

## 1. Cast

In [55]:
cast = spark.read.format("delta").load(f"s3a://tmdb-silver/cast/")
cast.limit(5).toPandas()


,movie_id,title,cast_id,character,credit_id,gender,id,name
0,19995,Avatar,242,Jake Sully,5602a8a7c3a3685532001c9a,2,65731,Sam Worthington
1,19995,Avatar,3,Neytiri,52fe48009251416c750ac9cb,1,8691,Zoe Saldana
2,19995,Avatar,25,Dr. Grace Augustine,52fe48009251416c750aca39,1,10205,Sigourney Weaver
3,19995,Avatar,4,Col. Quaritch,52fe48009251416c750ac9cf,2,32747,Stephen Lang
4,19995,Avatar,5,Trudy Chacon,52fe48009251416c750ac9d3,1,17647,Michelle Rodriguez


In [41]:
cast.printSchema()

root
 |-- movie_id: long (nullable = true)
 |-- title: string (nullable = true)
 |-- cast_id: integer (nullable = true)
 |-- character: string (nullable = true)
 |-- credit_id: string (nullable = true)
 |-- gender: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)



In [43]:
cast.count()

106300
movies_organizer --- 0.736297607421875 seconds ---


In [44]:
cast.filter(col("credit_id").isNull()).show()

+--------+-----+-------+---------+---------+------+---+----+
|movie_id|title|cast_id|character|credit_id|gender| id|name|
+--------+-----+-------+---------+---------+------+---+----+
+--------+-----+-------+---------+---------+------+---+----+



### Cast 0000000000 values:

In [45]:
cast.filter(col("credit_id")== "0000000000").show(5)

+--------+--------------------+-------+---------+----------+------+----+----+
|movie_id|               title|cast_id|character| credit_id|gender|  id|name|
+--------+--------------------+-------+---------+----------+------+----+----+
|  346081|           Sardaarji|   null|     null|0000000000|  null|null|null|
|   43630| The Helix... Loaded|   null|     null|0000000000|  null|null|null|
|   17644|Barney's Great Ad...|   null|     null|0000000000|  null|null|null|
|  114065|Down & Out With T...|   null|     null|0000000000|  null|null|null|
|  102840|  Sex With Strangers|   null|     null|0000000000|  null|null|null|
+--------+--------------------+-------+---------+----------+------+----+----+
only showing top 5 rows



In [46]:
cast.filter(col("credit_id")== "0000000000").count()

43

## 2. Crew

In [47]:
crew = spark.read.format("delta").load(f"s3a://tmdb-silver/crew/")
crew.cache()
crew.limit(5).toPandas()


,movie_id,title,credit_id,department,gender,id,job,name
0,19995,Avatar,52fe48009251416c750aca23,Editing,0,1721,Editor,Stephen E. Rivkin
1,19995,Avatar,539c47ecc3a36810e3001f87,Art,2,496,Production Design,Rick Carter
2,19995,Avatar,54491c89c3a3680fb4001cf7,Sound,0,900,Sound Designer,Christopher Boyes
3,19995,Avatar,54491cb70e0a267480001bd0,Sound,0,900,Supervising Sound Editor,Christopher Boyes
4,19995,Avatar,539c4a4cc3a36810c9002101,Production,1,1262,Casting,Mali Finn


In [48]:
crew.printSchema()

root
 |-- movie_id: long (nullable = true)
 |-- title: string (nullable = true)
 |-- credit_id: string (nullable = true)
 |-- department: string (nullable = true)
 |-- gender: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- name: string (nullable = true)



In [49]:
crew.count()

129609

In [50]:
crew.filter(col("credit_id")== "0000000000").show()

+--------+--------------------+----------+----------+------+----+----+----+
|movie_id|               title| credit_id|department|gender|  id| job|name|
+--------+--------------------+----------+----------+------+----+----+----+
|  137955|           Crowsnest|0000000000|      null|  null|null|null|null|
|  361505|Me You and Five B...|0000000000|      null|  null|null|null|null|
|   55831|  Boynton Beach Club|0000000000|      null|  null|null|null|null|
|  357441|   Karachi se Lahore|0000000000|      null|  null|null|null|null|
|  447027|     Running Forever|0000000000|      null|  null|null|null|null|
|  114065|Down & Out With T...|0000000000|      null|  null|null|null|null|
|  371085|           Sharkskin|0000000000|      null|  null|null|null|null|
|   19615|           Flying By|0000000000|      null|  null|null|null|null|
|  279759| Harrison Montgomery|0000000000|      null|  null|null|null|null|
|   43630| The Helix... Loaded|0000000000|      null|  null|null|null|null|
|   48382|Th

## 3. Movies

In [51]:
movies = spark.read.format("delta").load(f"s3a://tmdb-silver/movies/")
movies.limit(5).toPandas()

,movie_id,title,budget,homepage,original_language,original_title,overview,popularity,release_date,revenue,runtime,status,tagline,vote_average,vote_count
0,19995,Avatar,237000000.0,http://www.avatarmovie.com/,en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.",150.437576,2009-12-10,2.787965e+09,162,Released,Enter the World of Pandora.,7.2,11800
1,285,Pirates of the Caribbean: At World's End,300000000.0,http://disney.go.com/disneypictures/pirates/,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems.",139.082611,2007-05-19,9.610000e+08,169,Released,"At the end of the world, the adventure begins.",6.9,4500
2,206647,Spectre,245000000.0,http://www.sonypictures.com/movies/spectre/,en,Spectre,"A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. While M battles political forces to keep the secret service alive, Bond peels back the layers of deceit to reveal the terrible truth behind SPECTRE.",107.376785,2015-10-26,8.806746e+08,148,Released,A Plan No One Escapes,6.3,4466
3,49026,The Dark Knight Rises,250000000.0,http://www.thedarkknightrises.com/,en,The Dark Knight Rises,"Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's crimes to protect the late attorney's reputation and is subsequently hunted by the Gotham City Police Department. Eight years later, Batman encounters the mysterious Selina Kyle and the villainous Bane, a new terrorist leader who overwhelms Gotham's finest. The Dark Knight resurfaces to protect a city that has branded him an enemy.",112.312950,2012-07-16,1.084939e+09,165,Released,The Legend Ends,7.6,9106
4,49529,John Carter,260000000.0,http://movies.disney.com/john-carter,en,John Carter,"John Carter is a war-weary, former military captain who's inexplicably transported to the mysterious and exotic planet of Barsoom (Mars) and reluctantly becomes embroiled in an epic conflict. It's a world on the brink of collapse, and Carter rediscovers his humanity when he realizes the survival of Barsoom and its people rests in his hands.",43.926994,2012-03-07,2.841391e+08,132,Released,"Lost in our world, found in another.",6.1,2124


In [52]:
movies.printSchema()

root
 |-- movie_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- budget: double (nullable = true)
 |-- homepage: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: float (nullable = true)
 |-- release_date: date (nullable = true)
 |-- revenue: double (nullable = true)
 |-- runtime: integer (nullable = true)
 |-- status: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- vote_average: float (nullable = true)
 |-- vote_count: integer (nullable = true)



In [53]:
movies.count()

4803

## 4. Genres

In [16]:
genres = spark.read.format("delta").load("s3a://tmdb-silver/genres/")
genres.limit(5).toPandas()

,movie_id,id,name
0,19995,28,Action
1,19995,12,Adventure
2,19995,14,Fantasy
3,19995,878,Science Fiction
4,285,12,Adventure


In [17]:
genres.printSchema()

root
 |-- movie_id: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)



In [18]:
genres.filter(col("id").isNull()).show()

+--------+---+----+
|movie_id| id|name|
+--------+---+----+
+--------+---+----+



In [19]:
genres.filter(col("id")=="-9999").show()

+--------+-----+----+
|movie_id|   id|name|
+--------+-----+----+
|  191229|-9999|null|
|  206412|-9999|null|
|  371085|-9999|null|
|  346081|-9999|null|
|  335874|-9999|null|
|  281189|-9999|null|
|  137955|-9999|null|
|  357834|-9999|null|
|  380097|-9999|null|
|  282128|-9999|null|
|  162396|-9999|null|
|  328307|-9999|null|
|   48382|-9999|null|
|  126186|-9999|null|
|  198370|-9999|null|
|  325579|-9999|null|
|  294550|-9999|null|
|  219716|-9999|null|
|  279759|-9999|null|
|  176074|-9999|null|
+--------+-----+----+
only showing top 20 rows



In [20]:
genres.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in genres.columns]).show()

+--------+---+----+
|movie_id| id|name|
+--------+---+----+
|       0|  0|  28|
+--------+---+----+



# 5. Keywords

In [21]:
keywords = spark.read.format("delta").load("s3a://tmdb-silver/keywords/")
keywords.limit(5).toPandas()

,movie_id,id,name
0,19995,1463,culture clash
1,19995,2964,future
2,19995,3386,space war
3,19995,3388,space colony
4,19995,3679,society


In [22]:
keywords.printSchema()

root
 |-- movie_id: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)



In [23]:
keywords.filter(col("id").isNull()).show()

+--------+---+----+
|movie_id| id|name|
+--------+---+----+
+--------+---+----+



In [24]:
keywords.filter(col("id")=="-9999").show()

+--------+-----+----+
|movie_id|   id|name|
+--------+-----+----+
|    1735|-9999|null|
|   79698|-9999|null|
|   37786|-9999|null|
|   49852|-9999|null|
|  109418|-9999|null|
|   14177|-9999|null|
|   41283|-9999|null|
|  190955|-9999|null|
|  180296|-9999|null|
|  173931|-9999|null|
|   66607|-9999|null|
|   20542|-9999|null|
|  226486|-9999|null|
|   15173|-9999|null|
|   16643|-9999|null|
|  326284|-9999|null|
|   43418|-9999|null|
|   32316|-9999|null|
|   15365|-9999|null|
|  396152|-9999|null|
+--------+-----+----+
only showing top 20 rows



# 6. Production_Companies

In [34]:
production_companies = spark.read.format("delta").load("s3a://tmdb-silver/production_companies/")
production_companies.cache()
production_companies.limit(5).toPandas()

2022-11-12 19:20:33,667 WARN execution.CacheManager: Asked to cache already cached data.


,movie_id,id,name
0,19995,289,Ingenious Film Partners
1,19995,306,Twentieth Century Fox Film Corporation
2,19995,444,Dune Entertainment
3,19995,574,Lightstorm Entertainment
4,285,2,Walt Disney Pictures


In [35]:
production_companies.printSchema()

root
 |-- movie_id: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)



In [36]:
production_companies.filter(col("id").isNull()).show()

+--------+---+----+
|movie_id| id|name|
+--------+---+----+
+--------+---+----+



In [37]:
production_companies.filter(col("id") == "-9999").show()

+--------+-----+----+
|movie_id|   id|name|
+--------+-----+----+
|   10930|-9999|null|
|   58051|-9999|null|
|  103903|-9999|null|
|    2771|-9999|null|
|  206412|-9999|null|
|  371085|-9999|null|
|  191229|-9999|null|
|  112430|-9999|null|
|   62008|-9999|null|
|   75033|-9999|null|
|   10188|-9999|null|
|    3028|-9999|null|
|  183894|-9999|null|
|   19615|-9999|null|
|   20337|-9999|null|
|   70006|-9999|null|
|   12621|-9999|null|
|   14631|-9999|null|
|   22820|-9999|null|
|   13066|-9999|null|
+--------+-----+----+
only showing top 20 rows



# 7. Production_Countries

In [29]:
production_countries = spark.read.format("delta").load("s3a://tmdb-silver/production_countries/")
production_countries.limit(5).toPandas()

,movie_id,iso_3166_1,name
0,19995,US,United States of America
1,19995,GB,United Kingdom
2,285,US,United States of America
3,206647,GB,United Kingdom
4,206647,US,United States of America


In [30]:
production_countries.printSchema()

root
 |-- movie_id: string (nullable = true)
 |-- iso_3166_1: string (nullable = true)
 |-- name: string (nullable = true)



In [31]:
production_countries.filter(col("iso_3166_1").isNull()).show()

+--------+----------+----+
|movie_id|iso_3166_1|name|
+--------+----------+----+
+--------+----------+----+



In [32]:
production_countries.filter(col("iso_3166_1") == "XX").show()

+--------+----------+----+
|movie_id|iso_3166_1|name|
+--------+----------+----+
|   24113|        XX|null|
|   36597|        XX|null|
|  150211|        XX|null|
|  281189|        XX|null|
|   16433|        XX|null|
|  170480|        XX|null|
|   30979|        XX|null|
|  325123|        XX|null|
|  219716|        XX|null|
|  371085|        XX|null|
|  323270|        XX|null|
|  206412|        XX|null|
|  323271|        XX|null|
|  320435|        XX|null|
|   29731|        XX|null|
|  279759|        XX|null|
|   65448|        XX|null|
|   77332|        XX|null|
|   43653|        XX|null|
|   84659|        XX|null|
+--------+----------+----+
only showing top 20 rows



In [ ]:
# 8. Spoken_Languages

In [38]:
spoken_languages = spark.read.format("delta").load("s3a://tmdb-silver/spoken_languages/")
spoken_languages.limit(5).toPandas()

,movie_id,iso_639_1,name
0,19995,en,English
1,19995,es,Español
2,285,en,English
3,206647,fr,Français
4,206647,en,English


In [39]:
spoken_languages.printSchema()

root
 |-- movie_id: string (nullable = true)
 |-- iso_639_1: string (nullable = true)
 |-- name: string (nullable = true)



In [40]:
spoken_languages.filter(col("iso_639_1").isNull()).show()

+--------+---------+----+
|movie_id|iso_639_1|name|
+--------+---------+----+
+--------+---------+----+



In [41]:
spoken_languages.filter(col("iso_639_1") == "XX").show()

+--------+---------+----+
|movie_id|iso_639_1|name|
+--------+---------+----+
|  293644|       XX|null|
|   37694|       XX|null|
|   48382|       XX|null|
|   60463|       XX|null|
|   70875|       XX|null|
|  206412|       XX|null|
|  310933|       XX|null|
|  357834|       XX|null|
|   41317|       XX|null|
|  459488|       XX|null|
|  109410|       XX|null|
|  114065|       XX|null|
|   66607|       XX|null|
|  331493|       XX|null|
|   38786|       XX|null|
|  325140|       XX|null|
|  219716|       XX|null|
|  312113|       XX|null|
|   84401|       XX|null|
|   36584|       XX|null|
+--------+---------+----+
only showing top 20 rows

